In [131]:
import torch

In [132]:
import onnx

In [133]:
###########加载动态输入dbnet onnx,不需要使用sim
onnx_model=onnx.load("../onnx_models/dbnet/dbnet_mobile_dynamic.onnx")

In [71]:
#########加载静态输入dbnet onnx,使用simplify
onnx_model=onnx.load("../onnx_models/dbnet/dbnet_mobile_static_sim.onnx")

In [134]:
print(onnx.checker.check_model(onnx_model))

None


In [135]:
onnx_model.graph.input

[name: "input"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_value: 1
      }
      dim {
        dim_value: 3
      }
      dim {
        dim_param: "input_dynamic_axes_1"
      }
      dim {
        dim_param: "input_dynamic_axes_2"
      }
    }
  }
}
]

In [136]:
onnx_model.graph.output

[name: "output"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_value: 1
      }
      dim {
        dim_value: 1
      }
      dim {
        dim_param: "output_dynamic_axes_1"
      }
      dim {
        dim_param: "output_dynamic_axes_2"
      }
    }
  }
}
]

In [137]:
#########################加载输入图像

In [138]:
import cv2

In [139]:
src_img="../../../BankRecogProj/Test/1.jpg"

In [140]:
src_img=cv2.imread(src_img)

In [141]:
h,w,_=src_img.shape

In [142]:
#######################输入预处理

In [143]:
import sys
sys.path.append("../../../BankRecogProj_20220506/dbnet")

In [144]:
from utils.tools import *

In [145]:
import yaml

In [146]:
stream = open("../../../BankRecogProj_20220506/dbnet/config_mobile.yaml",'r',encoding="utf-8")

In [147]:
config=yaml.load(stream)

In [148]:
config['test']['short_side']

320

In [149]:
image=resize_image(src_img,config['test']['short_side'])

In [150]:
image.shape

(320, 576, 3)

In [151]:
import torchvision.transforms as transforms

In [152]:
image=transforms.ToTensor()(image)

In [153]:
image = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])(image).unsqueeze(0)

In [154]:
################onnxruntime推理

In [155]:
import onnxruntime

In [156]:
ort_session = onnxruntime.InferenceSession("../onnx_models/dbnet/dbnet_mobile_dynamic.onnx")

In [157]:
for input_meta in ort_session.get_inputs():
    print(input_meta)

NodeArg(name='input', type='tensor(float)', shape=[1, 3, 'input_dynamic_axes_1', 'input_dynamic_axes_2'])


In [158]:
for output_meta in ort_session.get_outputs():
    print(output_meta)

NodeArg(name='output', type='tensor(float)', shape=[1, 1, 'output_dynamic_axes_1', 'output_dynamic_axes_2'])


In [159]:
######################输入tensor切换成numpy

In [160]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

In [161]:
ort_inputs={ort_session.get_inputs()[0].name:to_numpy(image)}

In [162]:
ort_outs=ort_session.run(["output"],ort_inputs)

In [163]:
pred_map=ort_outs[0]

In [164]:
pred_map.shape

(1, 1, 320, 576)

In [165]:
from utils.DB_postprocesss import *

In [166]:
params={
    'thresh':config['test']['thresh'],
    'box_thresh':config['test']['box_thresh'],
    'max_candidates':config['test']['max_candidates'],
              'is_poly':config['test']['is_poly'],
              'unclip_ratio':config['test']['unclip_ratio'],
              'min_size':config['test']['min_size']
    
}

In [167]:
dbprocess=DBPostProcess(params)

In [168]:
####根据原图和feature map的比例回归到原图上
scale = (src_img.shape[1] * 1.0 / pred_map.shape[3], src_img.shape[0] * 1.0 / pred_map.shape[2]) 

In [169]:
bbox_batch,score_batch=dbprocess(pred_map,[scale])

In [170]:
bboxs=bbox_batch[0]

In [171]:
for box in bboxs:
    src_img = cv2.drawContours(src_img.copy(), [box.reshape(4,2).astype(np.int)], -1, (0, 255, 0), 1)
cv2.imwrite("../../Output/db_img_r.jpg",src_img)

True